In [2]:
import keras
import h5py
import numpy as np
import keras.backend as K

In [3]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    return x_data, y_data

In [4]:
def data_preprocess(x_data):
    return x_data/255

In [5]:
def score(x,y,model):
  clean_label_p = np.argmax(model.predict(x), axis=1)
  class_accu = np.mean(np.equal(clean_label_p, y))*100
  print('Classification accuracy:', class_accu)
  return class_accu

load all the data

In [20]:
x_clean_test, y_clean_test = data_loader("drive/MyDrive/data/clean_test_data.h5")
x_clean_test = data_preprocess(x_clean_test)
x_clean_val, y_clean_val = data_loader("drive/MyDrive/data/clean_validation_data.h5")
x_clean_val = data_preprocess(x_clean_val)
x_sunglass_poison, y_sunglass_poison = data_loader("drive/MyDrive/data/sunglasses_poisoned_data.h5")
x_sunglass_poison = data_preprocess(x_sunglass_poison)
x_anonymous, y_anonymous = data_loader("drive/MyDrive/data/anonymous_1_poisoned_data.h5")
x_anonymous = data_preprocess(x_anonymous)
x_eyebrows, y_eyebrows = data_loader("drive/MyDrive/data/eyebrows_poisoned_data.h5")
x_eyebrows = data_preprocess(x_eyebrows)
x_lipstick, y_lipstick = data_loader("drive/MyDrive/data/lipstick_poisoned_data.h5")
x_lipstick = data_preprocess(x_lipstick)

repair functions 

In [ ]:
def deleteWeight(weights, index):
  #return two lists of weights, weight_choice and bias_choice
  #choice[0] is the original weights of the net
  #choice[i] is the weights after deleting i nodes by index
  #maximum of i is 60 (total 60 nodes in conv_3 layer)

  node_weight = np.array(weights[4])#original weights
  bias_weight = np.array(weights[5])#original bias

  weight_choice = []
  weight_choice.append(node_weight)
  bias_choice = []
  bias_choice.append(bias_weight)
  for i in range(60):
    temp = np.array(weight_choice[-1])
    temp[:,:,:,index[i]] = 0   #set weights 0
    weight_choice.append(temp)
    temp2 = np.array(bias_choice[-1])
    temp2[index[i]] = 0   #set bias 0
    bias_choice.append(temp2)
  
  return weight_choice, bias_choice

In [ ]:
def prune(badnetModel, weight_choice, bias_choice):
  model = badnetModel
  new_weights = np.array(model.get_weights())
  new_weights[4] = weight_choice[50]  #update new weights
  new_weights[5] = bias_choice[50]   #update new bias
  model.set_weights(new_weights)
  return model

In [72]:
def retrain(prunedModel, x_val, y_val):
  model = prunedModel

  layer = model.get_layer(name='conv_3')
  layer.trainable = False
  layer = model.get_layer(name='pool_3')
  layer.trainable = False
  opt = keras.optimizers.Adadelta(learning_rate=1)
  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer=opt, loss=loss)
  model.fit(x_clean_val,y_clean_val, epochs=10, batch_size=32)
 # opt = keras.optimizers.Adadelta(learning_rate=0.1)
 ## model.compile(optimizer=opt, loss=loss)
  #model.fit(x_clean_val,y_clean_val, epochs=5, batch_size=32)

  return model

In [ ]:
def repair(badnetModel, x_val, y_val):
  repairedModel = badnetModel
  x = x_val
  y = y_val

  #getting the output from 'pool_3' layer
  function = keras.backend.function(badnetModel.inputs, badnetModel.get_layer('pool_3').output)
  pool_3_output = function(x_val)
  #calculate the mean activation of each node
  activation = np.mean(pool_3_output, axis=(0,1,2))
  #get a list of node index by activation increasing order
  index = np.argsort(activation)

  #get the weights and bias lists after each deletion
  weight_choice, bias_choice = deleteWeight(badnetModel.get_weights(), index)
  #prune the model
  repairedModel = prune(repairedModel, weight_choice, bias_choice)
  #retrain the model with validation set
  repairedModel = retrain(repairedModel, x_val, y_val)

  return repairedModel

repair bad net models

In [46]:
#sunglasses_repaired = repair(keras.models.load_model("/content/drive/MyDrive/models/sunglasses_bd_net.h5"), x_clean_val, y_clean_val)
#sunglasses_repaired.save("/content/drive/MyDrive/repaired models/sunglasses_repaired.h5")

In [58]:
#anonymous_1_repaired = repair(keras.models.load_model("/content/drive/MyDrive/models/anonymous_1_bd_net.h5"), x_clean_val, y_clean_val)
#anonymous_1_repaired.save("/content/drive/MyDrive/repaired models/anonymous_1_repaired.h5")

In [77]:
#anonymous_2_repaired = repair(keras.models.load_model("/content/drive/MyDrive/models/anonymous_2_bd_net.h5"), x_clean_val, y_clean_val)
#anonymous_2_repaired.save("/content/drive/MyDrive/repaired models/anonymous_2_repaired.h5")

In [70]:
#multi_trigger_repaired = repair(keras.models.load_model("/content/drive/MyDrive/models/multi_trigger_multi_target_bd_net.h5"), x_clean_val, y_clean_val)
#multi_trigger_repaired.save("/content/drive/MyDrive/repaired models/multi_trigger_repaired.h5")